In [10]:
import json
import openai
import os
import pandas as pd
from pprint import pprint
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

In [11]:
src_train_path = "data/NCA/src-train.txt"
tgt_train_path = "data/NCA/tgt-train.txt"
src_val_path = "data/NCA/src-val.txt"
tgt_val_path = "data/NCA/tgt-val.txt"
src_test_path = "data/NCA/src-test.txt"
tgt_test_path = "data/NCA/tgt-test.txt"
openai.api_key = ""

In [12]:
encoding = tiktoken.get_encoding("cl100k_base")
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):

    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

In [19]:
with open(src_train_path, 'r') as src_train, open(tgt_train_path, 'r') as tgt_train:
    with open('data/NCA/tgt-train-summarized.txt', 'w', encoding='utf-8') as file1, open('data/NCA/src-train-summarized.txt', 'w', encoding='utf-8') as file2:
        count = 0
        for src, tgt in zip(src_train, tgt_train):
            if tgt.strip:
                count += 1
                print(count)
            if count > 80:
                continue
            response = openai.ChatCompletion.create(
              model="gpt-3.5-turbo-1106",
              messages=[
                  {'role': 'system', 'content': "You are a highly skilled doctor with the ability to perfectly summarize a patient's health information. Note: '<PAR>' indicates a new line or paragraph separation. The requirements of the summary as below: 1. **Content**: Your summary should exclusively contain the patient's gender, age, race, current diagnoses, medications, and lab test results. 2. **Format**: Craft the information into a cohesive and informative paragraph. 3. **Accuracy**: Ensure that no required items are omitted. Do not include any past medical history or surgical information; focus solely on the present visit. 4. **Data Availability**: If specific information for the requested items does not exist, clearly indicate this by stating '<item name> data not found' without quotation marks. 5. **Relevance**: Avoid any mention of hospitalization-related matters or details outside the current visit's scope. Your goal is to provide a clear, concise, and accurate summary of the patient's current health status based on the presented visit's information. Remember to maintain a professional and empathetic tone throughout your summary."},
                  {'role': 'user', 'content': tgt.strip()}
              ]
            )
            print(response["choices"][0]["message"]["content"])
            result_text = response["choices"][0]["message"]["content"]
            file1.write(result_text + '\n')
            file2.write(src.strip() + '\n')

1
The patient is a 76-year-old white female presenting with worsening fatigue. She has a history of three-vessel coronary artery disease, myocardial infarction, poorly controlled hypertension, and hyperlipidemia. Current medications include aspirin, ranitidine, docusate sodium, potassium chloride, acetaminophen, magnesium hydroxide, tramadol, bisacodyl, metoprolol, simvastatin, warfarin, lorazepam, amiodarone, furosemide, and lisinopril. Lab results from [2188/11/4] show low hemoglobin (7.4), low hematocrit (22.2), and other values within normal limits. The patient underwent three-vessel coronary artery bypass grafting and was started on diuretics, betablockers, and ACE-inhibitors post-surgery. She also experienced intermittant atrial fibrillation and was started on amiodarone and warfarin. A solitary right lung nodule was found, and a follow-up appointment with a CT scan was recommended. Discharge was to extended care facility for continued rehabilitation. Follow-up appointments with 

In [17]:
with open(src_val_path, 'r') as src_val, open(tgt_val_path, 'r') as tgt_val:
     with open('data/NCA/tgt-val-summarized.txt', 'w', encoding='utf-8') as file1, open('data/NCA/src-val-summarized.txt', 'w', encoding='utf-8') as file2:
        count = 0
        for src, tgt in zip(src_val, tgt_val):
            if tgt.strip:
                count += 1
                print(count)
            if count > 10:
                continue
            response = openai.ChatCompletion.create(
              model="gpt-3.5-turbo-1106",
              messages=[
                  {'role': 'system', 'content': "You are a highly skilled doctor with the ability to perfectly summarize a patient's health information. Note: '<PAR>' indicates a new line or paragraph separation. The requirements of the summary as below: 1. **Content**: Your summary should exclusively contain the patient's gender, age, race, current diagnoses, medications, and lab test results. 2. **Format**: Craft the information into a cohesive and informative paragraph. 3. **Accuracy**: Ensure that no required items are omitted. Do not include any past medical history or surgical information; focus solely on the present visit. 4. **Data Availability**: If specific information for the requested items does not exist, clearly indicate this by stating '<item name> data not found' without quotation marks. 5. **Relevance**: Avoid any mention of hospitalization-related matters or details outside the current visit's scope. Your goal is to provide a clear, concise, and accurate summary of the patient's current health status based on the presented visit's information. Remember to maintain a professional and empathetic tone throughout your summary."},
                  {'role': 'user', 'content': tgt.strip()}
              ]
            )
       
            result_text = response["choices"][0]["message"]["content"]
            file1.write(result_text + '\n')
            file2.write(src.strip() + '\n')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [18]:
with open(src_test_path, 'r') as src_test, open(tgt_test_path, 'r') as tgt_test:
     with open('data/NCA/tgt-test-summarized.txt', 'w', encoding='utf-8') as file1, open('data/NCA/src-test-summarized.txt', 'w', encoding='utf-8') as file2:
        count = 0
        for src, tgt in zip(src_test, tgt_test):
            if tgt.strip:
                count += 1
                print(count)
            if count > 10:
                continue
            response = openai.ChatCompletion.create(
              model="gpt-3.5-turbo-1106",
              messages=[
                  {'role': 'system', 'content': "You are a highly skilled doctor with the ability to perfectly summarize a patient's health information. Note: '<PAR>' indicates a new line or paragraph separation. The requirements of the summary as below: 1. **Content**: Your summary should exclusively contain the patient's gender, age, race, current diagnoses, medications, and lab test results. 2. **Format**: Craft the information into a cohesive and informative paragraph. 3. **Accuracy**: Ensure that no required items are omitted. Do not include any past medical history or surgical information; focus solely on the present visit. 4. **Data Availability**: If specific information for the requested items does not exist, clearly indicate this by stating '<item name> data not found' without quotation marks. 5. **Relevance**: Avoid any mention of hospitalization-related matters or details outside the current visit's scope. Your goal is to provide a clear, concise, and accurate summary of the patient's current health status based on the presented visit's information. Remember to maintain a professional and empathetic tone throughout your summary."},
                  {'role': 'user', 'content': tgt.strip()}
              ]
            )
       
            result_text = response["choices"][0]["message"]["content"]
            file1.write(result_text + '\n')
            file2.write(src.strip() + '\n')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [26]:
with open("data/NCA/src-test-summarized.txt", 'r') as src_val, open("data/NCA/src-test_1.txt", 'r') as src_val_2:
    count = 0
    for line1, line2 in zip(src_val, src_val_2):
        if count <= 10:
            count += 1
            print(count)
        line1 = line1.strip()
        line2 = line2.strip()

        if count > 10:
            continue
        if line1[:30] != line2[:30]:
            print("Not match")

1
2
3
4
5
6
7
8
9
10
